**Neural Network Optimisation - Keras Tuner Hyperband**

In [1]:
!pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 3.3 MB/s eta 0:00:00


In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, recall_score, f1_score
import pandas as pd
import numpy as np

import tensorflow as tf
import datetime as dt
import keras_tuner as kt

In [3]:
# Import our input dataset
all_tele_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/Resources/all_tele_main.csv')
all_tele_df = all_tele_df.sort_values(by=['datetime']).drop_duplicates(subset=['datetime', 'machineid'], keep='first').reset_index(drop=True)
all_tele_df.head()

<ipython-input-3-51b3cfc196db>:2: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  all_tele_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Project/Resources/all_tele_main.csv')


,datetime,machineid,machinemodel,machineage,volt,rotate,pressure,vibration,errorid,compmaint,compfail
0,2015-01-01 06:00:00,34,model4,10,181.859394,483.215401,82.463735,37.849856,0,comp2,0
1,2015-01-01 06:00:00,41,model4,9,162.314459,424.540949,129.113722,52.591978,0,0,0
2,2015-01-01 06:00:00,39,model4,0,150.138449,421.747438,100.174935,40.367037,0,0,0
3,2015-01-01 06:00:00,10,model3,10,158.421261,500.830885,119.750673,45.571344,0,0,0
4,2015-01-01 06:00:00,45,model3,14,191.649670,383.200891,85.630577,52.169106,0,0,0


In [4]:
all_tele_df.dtypes

datetime         object
machineid         int64
machinemodel     object
machineage        int64
volt            float64
rotate          float64
pressure        float64
vibration       float64
errorid          object
compmaint        object
compfail          int64
dtype: object

In [5]:
# filter for one machine only
all_tele_df = all_tele_df.loc[(all_tele_df['machineid'] == 99)]
all_tele_df.head()

,datetime,machineid,machinemodel,machineage,volt,rotate,pressure,vibration,errorid,compmaint,compfail
34,2015-01-01 06:00:00,99,model1,14,168.596133,384.747105,110.921131,41.944692,0,0,0
192,2015-01-01 07:00:00,99,model1,14,153.667693,441.288719,128.011446,39.271527,0,0,0
261,2015-01-01 08:00:00,99,model1,14,178.319255,512.612661,128.526147,42.975412,0,0,0
363,2015-01-01 09:00:00,99,model1,14,162.163821,493.356626,114.435754,40.044016,0,0,0
418,2015-01-01 10:00:00,99,model1,14,180.201336,550.512296,110.771891,43.054530,0,0,0


In [6]:
# Update date format ready for testing/training
all_tele_df['datetime'] = pd.to_datetime(all_tele_df['datetime'])
all_tele_df['datetime'] = all_tele_df['datetime'].astype(np.int64)
all_tele_df.dtypes


datetime          int64
machineid         int64
machinemodel     object
machineage        int64
volt            float64
rotate          float64
pressure        float64
vibration       float64
errorid          object
compmaint        object
compfail          int64
dtype: object

In [7]:
# use get_dummies to convert categorical variables
all_tele_df = pd.get_dummies(data=all_tele_df, columns=['machinemodel', 'errorid', 'compmaint'])
all_tele_df

,datetime,machineid,machineage,volt,rotate,pressure,vibration,compfail,machinemodel_model1,errorid_0,...,errorid_error2,errorid_error3,errorid_error4,errorid_error5,compmaint_0,compmaint_0,compmaint_comp1,compmaint_comp2,compmaint_comp3,compmaint_comp4
34,1420092000000000000,99,14,168.596133,384.747105,110.921131,41.944692,0,1,0,...,0,0,0,0,1,0,0,0,0,0
192,1420095600000000000,99,14,153.667693,441.288719,128.011446,39.271527,0,1,0,...,0,0,0,0,0,1,0,0,0,0
261,1420099200000000000,99,14,178.319255,512.612661,128.526147,42.975412,0,1,0,...,0,0,0,0,0,1,0,0,0,0
363,1420102800000000000,99,14,162.163821,493.356626,114.435754,40.044016,0,1,0,...,0,0,0,0,1,0,0,0,0,0
418,1420106400000000000,99,14,180.201336,550.512296,110.771891,43.054530,0,1,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
875670,1451613600000000000,99,14,166.719608,462.956239,101.251044,44.450916,0,1,0,...,0,0,0,0,0,1,0,0,0,0
875741,1451617200000000000,99,14,168.434193,462.373730,98.379062,35.391502,0,1,0,...,0,0,0,0,1,0,0,0,0,0
875801,1451620800000000000,99,14,191.505730,418.940906,101.018747,43.204243,0,1,0,...,0,0,0,0,0,1,0,0,0,0
875946,1451624400000000000,99,14,185.423810,416.284422,99.564561,38.006329,0,1,0,...,0,0,0,0,1,0,0,0,0,0


In [8]:
# split preprocessed data into feature and target arrays
y = all_tele_df.compfail.values
X = all_tele_df.drop(columns='compfail').values

# split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, stratify=y)

In [9]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])

    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=10,
        step=2), activation=activation, input_dim=len(X_train[0])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=10,
            step=2),
            activation=activation))

    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy", tf.keras.metrics.TruePositives()])

    return nn_model;



In [10]:
  # define function for f1_score
def f1_score(y_true, y_pred):
    return f1_score(y_true, (y_pred > 0.5).astype(int))

In [11]:
# define tuner model variables & objective
obj = [kt.Objective('val_true_positives', direction='max'), 'val_accuracy']
tuner = kt.Hyperband(
    create_model,
    objective= obj,
    max_epochs=100,
    hyperband_iterations=2
    )

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,epochs=100,validation_data=(X_test_scaled,y_test))

Trial 508 Complete [00h 01m 29s]
multi_objective: -0.9977179169654846

Best multi_objective So Far: -6.0
Total elapsed time: 01h 52m 23s


In [14]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'tanh',
 'first_units': 3,
 'num_layers': 3,
 'units_0': 5,
 'units_1': 3,
 'units_2': 7,
 'units_3': 7,
 'units_4': 9,
 'units_5': 1,
 'tuner/epochs': 12,
 'tuner/initial_epoch': 4,
 'tuner/bracket': 4,
 'tuner/round': 2,
 'tuner/trial_id': '0115'}

In [15]:
# Evaluate best model against full test data
best_model2 = tuner.get_best_models(1)[0]
model_loss, model_accuracy, metric = best_model2.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")
#print(f"{val_true_positives}")

69/69 - 0s - loss: 0.0033 - accuracy: 1.0000 - true_positives: 5.0000 - 458ms/epoch - 7ms/step
Loss: 0.003294557798653841, Accuracy: 1.0


In [16]:
# Making predictions using the testing data
predictions = best_model2.predict(X_test_scaled)
predictions_bool = np.argmax(predictions, axis=1)

# Calculating the confusion matrix
cm = confusion_matrix(y_test, predictions_bool)
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"]
)

# Calculating the accuracy score
acc_score = accuracy_score(y_test, predictions_bool)

# Displaying results
print("Confusion Matrix")
display(cm_df)
print(f"Accuracy Score : {acc_score}")
print("Classification Report")
print(classification_report(y_test, predictions_bool))

69/69 [==============================] - 0s 2ms/step
Confusion Matrix


,Predicted 0,Predicted 1
Actual 0,2186,0
Actual 1,5,0


Accuracy Score : 0.9977179370150616
Classification Report
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      2186
           1       0.00      0.00      0.00         5

    accuracy                           1.00      2191
   macro avg       0.50      0.50      0.50      2191
weighted avg       1.00      1.00      1.00      2191



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
